In [56]:
from google import genai
from google.genai import types
from pydantic import BaseModel
import os
from dotenv import load_dotenv  # Import this
import json
import re

# Load environment variables from the .env file
load_dotenv()

# --- API KEY CONFIGURATION ---


# 1. Get the API Key
api_key = os.environ.get("GEMINI_API_KEY")

# Optional: Get model from env, or default to a string
model_name = os.environ.get("model", "gemini-2.0-flash-exp")

# Debug print (you can remove this later)
print(f"Debug - Key found: {'Yes' if api_key else 'No'}")
print(f"Debug - Model: {model_name}")

if not api_key:
    print("Error: API Key not found. Check your .env file.")


Debug - Key found: Yes
Debug - Model: gemini-2.5-flash-lite


In [57]:
def load_system_prompt(filename="prompt.txt"):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            print(f"Debug - Loaded system prompt from {filename}")
            return f.read()
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return "You are a helpful assistant."  # Fallback prompt

prompt = load_system_prompt()
print(f"Debug - System Prompt: {prompt[:50]}...")  # Print first 50 chars

Debug - Loaded system prompt from prompt.txt
Debug - System Prompt:  """

You are a highly reliable and safety-focused...


In [64]:
# 4. handle input
# user_input = input("Enter the text message to analyze: ")
user_input = "Your package delivery failed. Please update your address at [www.bestwebsite.com] to avoid return to sender."
print(f"Debug - User Input: {user_input}")

Debug - User Input: Your package delivery failed. Please update your address at [www.bestwebsite.com] to avoid return to sender.


In [59]:
full_prompt = prompt.format(user_input)
# print(full_prompt)


In [60]:
clinet = genai.Client(api_key=api_key)

In [61]:
res=clinet.models._generate_content(
    model=model_name,
    contents=full_prompt,
)
print(res.text)

1. **Thought**: The message claims a failed package delivery and urges the recipient to update their address via a provided link. This is a common phishing tactic. The domain "www.bestwebsite.com" is generic and highly suspicious for a legitimate delivery service. Legitimate services typically use their official domain names. The urgency created by "avoid return to sender" is a tactic to pressure the user into clicking without thinking.

2. **Action**: This message exhibits several strong indicators of a scam. The suspicious URL, the pretext of a delivery issue, and the implied urgency are all hallmarks of phishing attempts designed to steal personal information.

3. **Final Answer**:
```json
{
  "label": "Scam",
  "reasoning": "The message claims a failed package delivery and provides a suspicious generic URL ('www.bestwebsite.com') for address updates. This is a common phishing tactic to trick users into clicking malicious links and revealing personal information. The urgency implied

In [62]:
def extract_json(text):
    """
    Finds the first '{' and the last '}' in a string 
    and parses it as JSON.
    """
    try:
        # Regex to find the JSON block inside the text
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
    except Exception as e:
        print(f"JSON Parsing Error: {e}")
    
    # Fallback if parsing fails
    return {
        "label": "Uncertain", 
        "reasoning": "Could not parse AI response",
        "intent": "Unknown",
        "risk_factors": ["Parsing Error"]
    }

In [63]:
parsed_output = extract_json(res.text)
print("Parsed Response:", parsed_output)
print("="*40)
print(parsed_output)
print(type(parsed_output))
print("="*40)

Parsed Response: {'label': 'Scam', 'reasoning': "The message claims a failed package delivery and provides a suspicious generic URL ('www.bestwebsite.com') for address updates. This is a common phishing tactic to trick users into clicking malicious links and revealing personal information. The urgency implied by 'avoid return to sender' is a red flag designed to bypass critical thinking.", 'intent': 'To phish for personal information (address, potentially payment details) or to redirect the user to a malicious website.', 'risk_factors': ['Suspicious and generic URL', 'Urgency/fear tactic (failed delivery, return to sender)', 'Unsolicited message about a delivery issue', 'Lack of personalization (no tracking number or specific delivery service mentioned)']}
{'label': 'Scam', 'reasoning': "The message claims a failed package delivery and provides a suspicious generic URL ('www.bestwebsite.com') for address updates. This is a common phishing tactic to trick users into clicking malicious l